# Taller de Futbol Analytics

![Selección](images/seleccion.jpg)

La idea de este taller es que vean parte de lo que se puede aprender en la facultad, en las carreras de Licenciatura en Ciencias Matemáticas y Ciencias de Datos de una manera interactiva. Nos centramos en el caso del fútbol, pero es interesante pensar que estos conceptos los pueden aplicar en cualquier cosa que les interese en su vida.

In [ ]:
# Primero importamos unas librerías de Python que vamos a usar a lo largo del notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipywidgets as widgets

from IPython.display import display
from scipy.stats import zscore
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from adjustText import adjust_text

In [ ]:
# Instalamos las librerías necesarias si no las tenemos instaladas

# !pip install adjustText
# !pip install plotly
# !pip install ipywidgets
# !pip install anywidgets

Para trabajar con los datos de los jugadores, los obtuvimos de dos páginas web de referencia en el ámbito de las estadísticas futbolísticas: FBRef y Transfermarkt

<img src="images/fbref.png" width=400pt> <img src="images/transfermarkt.png" width=280pt>

De FBRef obtuvimos las estadísticas de todos los jugadores en estas temporadas y de Transfermarkt obtuvimos su valor de mercado. Estos datos suelen ser de mucha importancia para los clubes de fútbol, ya que les interesa comprar jugadores que tengan buenas estadísticas y valgan poco dinero.

In [ ]:
# Obtenemos los datos desde el archivo df_reducido.csv
df = pd.read_csv("datasets_procesados/df_reducido.csv")
df_completo = pd.read_csv("datasets_procesados/df.csv")

Los datos los tenemos en un formato de tabla, donde cada fila es un jugador, y cada columna es algún dato sobre el mismo. Podemos obtener una visualización de una pequeña parte de esta **gigante** tabla con el comando `head`:

In [ ]:
# Podemos obtener una primera visualización de los datos viendo sus primeras 5 filas con el comando head()
display(df.head(), df_completo.head())

También podemos usar el comando `columns` para ver específicamente cuáles son los datos que tenemos sobre cada jugador:

In [ ]:
# Con este comando podemos ver las columnas e información de los datos que tenemos
display(df.columns, df_completo.columns)

Cómo podrán ver, tenemos un **montón** de información sobre cada uno de los jugadores (los cuales también son muchos). Entonces la pregunta lógica que uno se hace, y que se intenta responder en nuestras carreras, es **¿de qué manera podemos resumir esta información?** más allá, poniendose en el rol de un analista de datos de algún club: **¿de qué manera podemos sacar datos que nos sean de utilidad para la próxima compra de nuestro club?**

# Gráficos y visualización de datos

Lo primero que podríamos hacer es intentar tener una visualización directa de los datos en algún gráfico, para tener una idea de cómo se distribuyen las cosas. El problema es que no podemos graficar más de dos estadísticas juntas, así que hicimos un graficador interactivo para que ustedes puedan ir comparando las estadísticas que les parezca.

Por cada jugador, se hace un punto, que tiene color más claro a medida que más caro es el mismo.

In [ ]:
# Elegimos el dataframe con el que vamos a trabajar
df_grafico1 = df.copy()

# Filtrar jugadores con más de 90 minutos
df_grafico1 = df_grafico1[df_grafico1["Minutos"] > 90]

# Identificar columnas numéricas
numeric_columns = df_grafico1.select_dtypes(include=[np.number]).columns

# Crear widgets para seleccionar las columnas
x_dropdown = widgets.Dropdown(
    options=numeric_columns,
    value='Goles',
    description='Eje X:'
)

y_dropdown = widgets.Dropdown(
    options=numeric_columns,
    value='Asistencias',
    description='Eje Y:'
)

posiciones_dropdown = widgets.Dropdown(
    options=[
        ('Todos', 'all'),
        ('Delanteros', 'FW'),
        ('Mediocampistas', 'MF'),
        ('Defensores', 'DF'),
        ('Arqueros', 'GK')
    ],
    value='all',
    description='Posición:'
)


# Elegimos que columnas queremos que se normalicen sobre 90 minutos
normalizables = [
    'Goles', 'Asistencias', 'xG', 'xAG', 'npxG', 'Goles_Asistencias', 'xG_xAG',
    'Tiros', 'TirosAlArco', 'PasesClave', 'Entradas', 'Intercepciones', 'Bloqueos',
    'EntradasGanadas', 'AsistDesdeCruzados'
]

# Función para actualizar el gráfico
def update_graph(x_col, y_col, pos):

    df_filtrado = df_grafico1.copy()

    plt.figure(figsize=(10, 6))

    # Filtrar por posición si corresponde
    if pos != 'all':
        df_filtrado = df_filtrado[df_filtrado["Posición"] == pos]

    # Aplico normalización solo si corresponde
    x_data = df_filtrado[x_col]
    y_data = df_filtrado[y_col]

    if x_col in normalizables:
        x_data = 90 * x_data / df_filtrado['Minutos']
    if y_col in normalizables:
        y_data = 90 * y_data / df_filtrado['Minutos']

    # Scatter plot
    plt.scatter(
        x=x_data,
        y=y_data,
        c=np.log10(df_filtrado['ValorMercado'] + 1),  # Sumar 1 para evitar log(0)
        cmap='viridis',
        alpha=0.6,
        s=50
    )
    cbar = plt.colorbar()
    cbar.set_label('Log10(Valor de mercado €)', fontsize=10)

    # Etiquetas dinámicas
    x_label = f"{x_col} {'(cada 90 min)' if x_col in normalizables else ''}"
    y_label = f"{y_col} {'(cada 90 min)' if y_col in normalizables else ''}"

    # Títulos y etiquetas
    plt.title(f'{x_col} vs {y_col} - {pos if pos != "all" else "Todos"}', fontsize=14)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

# Conectar los widgets a la función
ui = widgets.VBox([posiciones_dropdown, x_dropdown, y_dropdown])
out = widgets.interactive_output(update_graph, {
    'x_col': x_dropdown,
    'y_col': y_dropdown,
    'pos': posiciones_dropdown
})

# Mostrar los widgets y el gráfico
display(ui, out)

Lo que vimos recién se denomina `scatter plot` o, en español, `diagrama de dispersión` y suele ser muy útil para resumir información cuando tenemos una gran cantidad de datos, pues nos ayuda a ver estadísticas de todos los jugadores de un solo vistazo, y también nos puede servir para hallar relaciones entre las mismas.

Por ejemplo: a lo mejor esperamos que si un jugador tiene más pases claves, esperamos que tenga más asistencias, o si tiene más tiros, esperamos que tenga más goles. A partir de cómo se distribuyen los puntos en el gráfico podemos inferir si eso es realmente verdad o no.

Podemos también ver el rendimiento de los jugadores en cuánto a sus goles, asistencias, goles esperados y asistencias esperados. Lo mostramos entonces en el siguiente gráfico:

In [ ]:
# Elegimos estadísticas y jugadores
estadisticas = ['Goles', 'Asistencias', 'xG', 'xAG']
jugadores_top10 = df.iloc[0:10]
x = np.arange(len(jugadores_top10))
bar_width = 0.2

# Elegimos una paleta de colores
colors = plt.get_cmap("Paired")(np.linspace(0, 1, len(estadisticas)))

# Preparamos el gráfico
fig, ax = plt.subplots(figsize=(12, 6))

# Dibujamos las barras
for i, (stat, color) in enumerate(zip(estadisticas, colors)):
    ax.bar(
        x + i * bar_width,
        jugadores_top10[stat],
        width=bar_width,
        label=stat,
        color=color
    )

# Etiquetas y estilo
ax.set_xticks(x + bar_width * (len(estadisticas)-1) / 2)
ax.set_xticklabels(jugadores_top10["Jugador"], rotation=45, ha='right')
ax.set_ylabel("Valor")
ax.set_title("Comparación de rendimiento (Top 10 jugadores más valiosos)")
ax.legend()
ax.grid(True, axis='y', linestyle='--', alpha=0.7)

# Mostramos el gráfico
plt.tight_layout()
plt.show()

![caros](images/caros.jpg)

Lo que hicimos recién se llama `bar plot` o `gráfico de barras` y nos ayuda a comparar rápidamente la cantidad de goles y asistencias de todos los jugadores.

En base al gráfico de recién, ¿creen que el  `xG` (goles esperados) y el  `xA` (asistencias esperadas) dan una buena idea de cuántos goles y asistencias se espera que un jugador haga?, es decir ¿se parecen para cada jugador su barra de `Goles` con la de `xG` y la de `Asistencias` con la de `xA`? ¿Cómo podrían explicar esto?

Ok, pero y si quisieramos comparar dos jugadores entre sí para ver cuál de ellos está teniendo una mejor temporada, cómo podríamos verlo de una forma rápida?

In [ ]:
# Seleccionamos el dataframe con el que vamos a trabajar
df_grafico2 = df.copy()

# Lo ordenamos alfabéticamente por el nombre del jugador
df_grafico2.sort_values("Jugador", inplace=True)

# Seleccionamos los datos que vamos a graficar
stats = [
    "Goles", "Asistencias", "Tiros", "TirosAlArco", "xG", "xAG",
    "PasesCompletados", "PasesClave", "DistanciaProgresiva", "Entradas", "Intercepciones"
]

# Armamos el dataframe con las columnas que nos interesan
df_grafico2 = df_grafico2[["Jugador"] + stats]

# Normalizamos los datos para que estén entre 0 y 1
scaler = MinMaxScaler()
df_grafico2[stats] = scaler.fit_transform(df_grafico2[stats])

# Función que extrae lista de valores normalizados
def valores(j):
    return df_grafico2.loc[df_grafico2["Jugador"] == j, stats].iloc[0].tolist()

# Obtenemos el listado de jugadores y dos jugadores iniciales para comparar
jugadores = df_grafico2["Jugador"]
j1, j2 = "Lionel Messi", "Harry Kane"

# Creamos el FigureWidget con las estadisticas del jugador 1
figw = go.FigureWidget(
    go.Scatterpolar(r=valores(j1), theta=stats, fill="toself", name=j1),
    layout=go.Layout(
        title=f"Comparación: {j1} vs {j2}",
        polar=dict(radialaxis=dict(visible=True, range=[0,1])),
        width=1000,
        height=600
    )
)

# Añadimos el segundo jugador al gráfico
figw.add_trace(go.Scatterpolar(r=valores(j2), theta=stats, fill="toself", name=j2))

# Creamos el menú desplegable para ambos jugadores
dd1 = widgets.Dropdown(options=jugadores, value=j1, description="Jugador 1")
dd2 = widgets.Dropdown(options=jugadores, value=j2, description="Jugador 2")

# Funcion que actualiza el gráfico al cambiar los jugadores
def actualizar(change):
    # trace 0
    figw.data[0].r = valores(dd1.value)
    figw.data[0].name = dd1.value
    # trace 1
    figw.data[1].r = valores(dd2.value)
    figw.data[1].name = dd2.value
    # título
    figw.layout.title.text = f"Comparación: {dd1.value} vs {dd2.value}"

# Actualizamos el gráfico al cambiar los jugadores
dd1.observe(actualizar, names="value")
dd2.observe(actualizar, names="value")

# Mostramos el gráfico interactivo
display(widgets.HBox([dd1, dd2]))
display(figw)

![Messi y Kane](images/messi_kane.jpg)

# Jugadores parecidos

En el fútbol muchas veces nos interesa cuando se nos va un jugador importante de nuestro club saber cuál es su posible mejor reemplazo. Acá probamos un enfoque bastante básico: dado un jugador, buscaremos cuáles son los jugadores que tienen estadísticas "más cercanas" a él, como opciones para reemplazarlo en caso de que se vaya de nuestro club

In [ ]:
# Nos quedamos con las columnas numéricas del dataframe completo
df_num = df_completo.select_dtypes(include=[np.number])
df_num[df_num.columns] = MinMaxScaler().fit_transform(df_num)

In [ ]:
# Definimos una función que nos devuelva, mediante knn, los k jugadores más parecidos para un jugador determinado

def jugadores_parecidos(nombre_jugador, cant_recomendaciones):
    # Obtenemos el índice del jugador
    ind_jugador = df[df["Jugador"]==nombre_jugador].index[0]

    # Inicializamos un NearestNeighbors que busque los n+1 jugadores más parecidos al buscado (porque incluye al jugador buscado)
    neighbors = NearestNeighbors(n_neighbors = cant_recomendaciones + 1).fit(df_num)

    # Obtenemos las distancias a esos jugadores y sus índices
    distances, indices = neighbors.kneighbors(df_num.iloc[[ind_jugador]])

    # Devolvemos el dataset evaluado en esos índices
    res = df.iloc[indices.flatten()]

    # Eliminamos de la lista al mismo jugador
    res = res.drop(ind_jugador, axis=0)
    
    return res

Lo que hace nuestra función `jugadores_parecidos` que creamos en la celda de arriba es precisamente lo que dijimos, mide de cierta forma los jugadores que tienen estadísticas más parecidas al nuestro y devuelve los primeros `n` que más se parezcan. Por ejemplo, si quisieramos ver los 10 jugadores más parecidos a Messi en su forma de jugar, podríamos hacer lo siguiente:

In [ ]:
# Obtenemos los 10 jugadores más parecidos a Lionel Messi
jugadores_parecidos("Lionel Messi", 10)

Y los invitamos a que experimenten un poco con esto y se fijen cuáles son los jugadores más parecidos a cualquiera de sus jugadores favoritos:

In [ ]:
jugador = ""
jugadores_parecidos(jugador, 10)

# Mejores jugadores

Podemos también fijarnos cuáles son los mejores jugadores del mundo en cada posición. Esto por ejemplo le podría llegar a interesar a la FIFA para armar un 11 ideal que premiar a fin de año:

In [ ]:
# Separamos el dataset en 4, uno por cada posición
df_delanteros = df_completo[df_completo["stats_Pos"] == "FW"]
df_mediocampistas = df_completo[df_completo["stats_Pos"] == "MF"]
df_defensores = df_completo[df_completo["stats_Pos"] == "DF"]
df_arqueros = df_completo[df_completo["stats_Pos"] == "GK"]

In [ ]:
def mejores_jugadores(posicion, cantidad, nacionalidad = None):

    # Decidimos con qué dataframe trabajar según la posición
    if posicion == "delanteros":
        df_pos = df_delanteros
        stats = df_num.columns

    elif posicion == "mediocampistas":
        df_pos = df_mediocampistas
        stats = df_num.columns

    elif posicion == "defensores":
        df_pos = df_defensores
        stats = df_num.columns

    elif posicion == "arqueros":
        df_pos = df_arqueros
        stats = df_num.columns
        
    elif posicion == "todos":
        df_pos = df.copy()
        stats = df_num.columns

    if nacionalidad is not None:
        df_pos = df_pos[df_pos["stats_Nation"].str.contains(nacionalidad, na=False)]

    # Aplicamos Z-Score a las columnas
    df_pos_zcores = df_pos[stats].apply(zscore)

    # Calculamos el puntaje total sumando los Z-Scores de cada estadística, ya que todas las columnas son positivas
    df_pos_zcores["score"] = df_pos_zcores[stats].sum(axis=1)

    # Ordenamos el dataframe por el puntaje total
    df_pos_zcores = df_pos_zcores.sort_values(by="score", ascending=False)

    # Ordenamos el dataframe original por el puntaje total
    mejores_jugadores = df_pos_zcores.head(cantidad)

    # Devolvemos el dataframe original con los mejores jugadores
    return df.loc[mejores_jugadores.index]

Probemos entonces fijarnos cuál sería el 11 ideal del mundo, por ejemplo, si pedimos una formación 4-3-3 (esto, como todo lo pueden ajustar a su gusto)

In [ ]:
# 11 ideal según estadísticas
# Formación: 4-3-3

cant_defensores = 4
cant_mediocampistas = 3 
cant_atacantes = 3

display(
    mejores_jugadores("delanteros", cant_atacantes),
    mejores_jugadores("mediocampistas", cant_mediocampistas),
    mejores_jugadores("defensores", cant_defensores),
    mejores_jugadores("arqueros", 1)
)

#Obs: El dataset que conseguimos, no tiene estadísticas específicas de los arqueros (paradas, arcos en 0, penales atajados, etc.)
#      lo cual explica por qué el resultado no es el Dibu Martínez.

Entonces, pareciera ser que la mejor formación que uno podría crear es:

![](images/11_mundo.png)

Por supuesto, cada uno juzgará que tan confiable o no le parece esto, pero eso es lo entretenido, si nos parece que nos equivocamos, podemos buscar mejores maneras de determinar cuáles son los mejores jugadores. No existe una sola, así que en ese sentido, tenemos total libertad para experimentar y eventualmente llegar a mejores resultados.

Por último, nos atrevimos a tomar el rol de Scaloni y ver cuál podría llegar a ser el mejor 11 argentino usando el mismo método de antes, y lo que resultó fue lo siguiente:

In [ ]:
# Mejor 11 ideal de jugadores argentinos, según estadísticas
# Formación: 4-3-3

display(
    mejores_jugadores("delanteros", 3, "ARG"),
    mejores_jugadores("mediocampistas", 3, "ARG"),
    mejores_jugadores("defensores", 4, "ARG"),
    mejores_jugadores("arqueros", 1, "ARG")
)

![](images/11_argentina.png)

Nuevamente, alguno quizás puede interpretar que los resultados obtenidos son polémicos, o quizás... ¿será que tenemos que levantar el teléfono de Leo Scaloni y pedir a Balerdi de 2? Lo dejamos a libre interpretación.

**Muchas gracias por su atención!**

Santi Oviedo y Faus Martínez.

![](images/logo_dm.jpg)